In [ ]:
!pip install requests
!pip install beautifulsoup4
!pip install selenium
!pip install pandas
!pip install openpyxl
!pip install lxml
!pip install html5lib

ERROR: Could not find a version that satisfies the requirement re (from versions: none)
ERROR: No matching distribution found for re


In [26]:
#books實體書30天月暢銷榜單(每日更新)
import requests
from bs4 import BeautifulSoup
import csv
import os
import re
from datetime import datetime

# 主網頁 URL
base_url = 'https://www.books.com.tw/web/sys_saletopb/books/'

# 各分類的 URL 後綴
categories = {
    '總榜': '',
}
# 獲取當天日期
today_date = datetime.now().strftime('%Y%m%d')
# 獲取今天日期作為欄位名稱（格式：月/日，例如：11/14）
today_column = datetime.now().strftime('%-m/%-d') if os.name != 'nt' else datetime.now().strftime('%#m/%#d')

# 儲存目錄與檔案名稱
output_dir = r'C:/Users/HP/Desktop/project/book-ranking/ranking_result/bookscom'
log_dir = r'C:/Users/HP/Desktop/project/book-ranking/實體書排行榜code/log'
os.makedirs(output_dir, exist_ok=True)
os.makedirs(log_dir, exist_ok=True)
csv_file_path = os.path.join(output_dir, f'books_all_categories_{today_date}.csv')
log_file_path = os.path.join(log_dir, f'books_plog_{today_date}.txt')

# 儲存所有數據的列表
all_books_data = []

# 初始化統計變數
total_books = 0
total_successful_books = 0
total_failed_books = 0

# 開始抓取
with open(log_file_path, 'w', encoding='utf-8') as log_file:
    log_entries = []
    log_entries.append("開始抓取所有分類的書籍數據...\n")
    print("開始抓取所有分類的書籍數據...")

    for category, suffix in categories.items():
        url = base_url + suffix
        category_total = 0
        category_successful = 0
        category_failed = 0

        # 發送請求並解析 HTML
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')

        # 選擇所有書籍的條目
        books = soup.select('ul.clearfix li.item')  # 更精確選擇榜單書籍
        category_total = len(books)
        total_books += category_total

        log_entries.append(f"開始解析分類: {category}")
        print(f"開始解析分類: {category}")

        # 逐一解析每本書籍的資訊
        for index, book in enumerate(books, start=1):
            try:
                # 排名
                rank_element = book.select_one('.no')
                rank = rank_element.text.strip() if rank_element and rank_element.text.strip().isdigit() else None

                # 書名
                title_element = book.select_one('h4 a')
                title = title_element.text.strip() if title_element else None

                # 正規化書名（去掉標點符號和空格）
                processed_title = re.sub(r'[^\w]', '', title) if title else ''

                # 書籍鏈結
                book_link = title_element['href'] if title_element else None

                # 平台產品編號（從URL提取）
                production_id = book_link.split('/products/')[-1].split('?')[0] if book_link else None

                # 作者
                author_element = book.select_one('.msg li a')
                author = author_element.text.strip() if author_element else ''

                # 過濾條件：確保所有字段有效
                if rank and title and book_link and production_id:
                    # 按照格式要求存入資料：
                    # ISBN, production_id, title, processed_title, Publisher, author, translator, 
                    # original_title, publish_date, fixed_price, category, url, Day_counts, 今天日期
                    all_books_data.append([
                        '',  # ISBN (空)
                        production_id,  # production_id
                        title,  # title
                        processed_title,  # processed_title (去掉標點符號和空格)
                        '',  # Publisher (空)
                        author,  # author
                        '',  # translator (空)
                        '',  # original_title (空)
                        '',  # publish_date (空)
                        '',  # fixed_price (空)
                        '',  # category (空 - 不爬取)
                        book_link,  # url
                        '',  # Day_counts (空)
                        rank  # 今天的排名
                    ])
                    category_successful += 1
                    total_successful_books += 1

                    # 日誌記錄成功條目
                    success_message = f"[成功] 分類: {category}, 書籍 {index}: 排名: {rank}, 產品編號: {production_id}, 書名: {title}, 作者: {author}, 鏈結: {book_link}"
                    print(success_message)
                    log_entries.append(success_message)
                else:
                    category_failed += 1
                    total_failed_books += 1
                    error_message = f"[跳過] 分類: {category}, 書籍 {index}: 資料不完整，已略過。"
                    print(error_message)
                    log_entries.append(error_message)

            except Exception as e:
                category_failed += 1
                total_failed_books += 1
                error_message = f"[錯誤] 分類: {category}, 書籍 {index}: 解析失敗，錯誤訊息: {e}"
                print(error_message)
                log_entries.append(error_message)

        # 計算分類成功率
        category_success_rate = (category_successful / category_total * 100) if category_total > 0 else 0
        log_entries.append(f"\n分類: {category} 統計結果:")
        log_entries.append(f"總抓取筆數: {category_total}")
        log_entries.append(f"成功筆數: {category_successful}")
        log_entries.append(f"失敗筆數: {category_failed}")
        log_entries.append(f"成功率: {category_success_rate:.2f}%\n")

        print(f"分類: {category} 解析完成。")

    # 總計統計
    total_success_rate = (total_successful_books / total_books * 100) if total_books > 0 else 0
    log_entries.append("總計統計結果:")
    log_entries.append(f"總抓取筆數: {total_books}")
    log_entries.append(f"成功筆數: {total_successful_books}")
    log_entries.append(f"失敗筆數: {total_failed_books}")
    log_entries.append(f"成功率: {total_success_rate:.2f}%\n")

    print("\n總計統計結果:")
    print(f"總抓取筆數: {total_books}")
    print(f"成功筆數: {total_successful_books}")
    print(f"失敗筆數: {total_failed_books}")
    print(f"成功率: {total_success_rate:.2f}%\n")

    # 將日誌記錄寫入日誌文件
    log_file.write("\n".join(log_entries))

# 將所有數據保存為一個 CSV 檔案
with open(csv_file_path, 'w', newline='', encoding='utf-8-sig') as file:
    writer = csv.writer(file)
    # 寫入標題行，最後一欄是今天的日期
    header = ['ISBN', 'production_id', 'title', 'processed_title', 'Publisher', 'author', 
              'translator', 'original_title', 'publish_date', 'fixed_price', 
              'category', 'url', 'Day_counts', today_column]
    writer.writerow(header)
    writer.writerows(all_books_data)

# 結束提示
print(f"所有分類的書籍數據已保存到 {csv_file_path}")
print(f"日誌已保存到 {log_file_path}")


開始抓取所有分類的書籍數據...
開始解析分類: 總榜
[成功] 分類: 總榜, 書籍 1: 排名: 1, 產品編號: 0010868562, 書名: 國寶（上下套書）, 作者: 吉田修一, 鏈結: https://www.books.com.tw/products/0010868562?loc=P_0003_001
[成功] 分類: 總榜, 書籍 2: 排名: 2, 產品編號: 0011033182, 書名: 永遠記得：男孩、鼴鼠、狐狸、馬與風暴【暢銷經典全新續集】（限量燙金書籤版）, 作者: 查理．麥克斯, 鏈結: https://www.books.com.tw/products/0011033182?loc=P_0003_002
[成功] 分類: 總榜, 書籍 3: 排名: 3, 產品編號: 0011034764, 書名: Gemini x NotebookLM 領軍：Nano Banana x Imagen x Veo x Gem x Gemini Live - Google 多模態 AI 工作流, 作者: 洪錦魁, 鏈結: https://www.books.com.tw/products/0011034764?loc=P_0003_003
[成功] 分類: 總榜, 書籍 4: 排名: 4, 產品編號: 0011034787, 書名: 活出你的本來面目：獻給現代人的榮格心靈指南：在有限的人生裡，活出此生的真正意義（首刷限定附贈「邁向完整的你」書卡組）, 作者: 鐘穎, 鏈結: https://www.books.com.tw/products/0011034787?loc=P_0003_004
[成功] 分類: 總榜, 書籍 5: 排名: 5, 產品編號: 0011036755, 書名: Babe Fantasy 寶兒寫真書 限量寶盒版【博客來獨家】, 作者: 寶兒, 鏈結: https://www.books.com.tw/products/0011036755?loc=P_0003_005
[成功] 分類: 總榜, 書籍 6: 排名: 6, 產品編號: 0011034598, 書名: ONE PIECE航海王 112, 作者: 尾田榮一郎, 鏈結: https://www.books.com.tw/products/0011034598?loc=P_

In [ ]:
#kingstone實體書30天月暢銷榜單(每日更新) - 僅總榜月排行
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
import csv
import os
import re
from datetime import datetime
import time

# 主網頁 URL - 總榜月排行
url = 'https://www.kingstone.com.tw/bestseller/best/book?ranktype=m'

# 獲取當天日期
today_date = datetime.now().strftime('%Y%m%d')
# 獲取今天日期作為欄位名稱（格式：月/日，例如：11/17）
today_column = datetime.now().strftime('%-m/%-d') if os.name != 'nt' else datetime.now().strftime('%#m/%#d')

# 儲存目錄與檔案名稱
output_dir = r'C:/Users/HP/Desktop/project/book-ranking/ranking_result/kingstone'
log_dir = r'C:/Users/HP/Desktop/project/book-ranking/實體書排行榜code/log'
os.makedirs(output_dir, exist_ok=True)
os.makedirs(log_dir, exist_ok=True)
csv_file_path = os.path.join(output_dir, f'kingstone_all_categories_{today_date}.csv')
log_file_path = os.path.join(log_dir, f'kingstone_plog_{today_date}.txt')

# 儲存所有數據的列表
all_books_data = []

# 初始化統計變數
total_books = 0
total_successful_books = 0
total_failed_books = 0

# 設定 Chrome 選項
options = Options()
# options.add_argument("--headless")  # 如需無頭模式可以取消註釋
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")

# 啟動瀏覽器
driver = webdriver.Chrome(options=options)

# 開始抓取
with open(log_file_path, 'w', encoding='utf-8') as log_file:
    log_entries = []
    log_entries.append("開始抓取金石堂總榜月排行書籍數據...\n")
    print("開始抓取金石堂總榜月排行書籍數據...")
    
    try:
        # 訪問頁面
        driver.get(url)
        print("正在載入頁面...")
        
        # 等待書籍列表加載
        wait = WebDriverWait(driver, 15)
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "li.modProList")))
        time.sleep(2)  # 額外等待確保所有內容加載完成
        
        # 滾動頁面以觸發懶加載（Lazy Loading）
        print("正在滾動頁面以加載所有書籍...")
        last_height = driver.execute_script("return document.body.scrollHeight")
        
        for i in range(5):  # 最多滾動5次
            # 滾動到頁面底部
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)  # 等待內容加載
            
            # 計算新的頁面高度
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break  # 如果高度沒變，說明已經到底了
            last_height = new_height
        
        # 滾動回頂部
        driver.execute_script("window.scrollTo(0, 0);")
        time.sleep(1)
        
        # 選擇所有書籍的條目
        books = driver.find_elements(By.CSS_SELECTOR, "li.modProList")
        total_books = len(books)
        
        log_entries.append(f"開始解析金石堂總榜月排行，共找到 {total_books} 本書")
        print(f"開始解析金石堂總榜月排行，共找到 {total_books} 本書")
    
        # 逐一解析每本書籍的資訊
        for index, book in enumerate(books, start=1):
            try:
                # 排名
                rank = None
                try:
                    rank_element = book.find_element(By.CSS_SELECTOR, '.modProRank span')
                    rank_text = rank_element.text.strip()
                    rank = rank_text if rank_text.isdigit() else None
                except Exception as e:
                    print(f"[除錯] 書籍 {index} 排名提取失敗: {e}")
                
                # 書名
                title = None
                title_element = None
                try:
                    title_element = book.find_element(By.CSS_SELECTOR, '.modProName a')
                    title = title_element.text.strip() if title_element else None
                except Exception as e:
                    print(f"[除錯] 書籍 {index} 書名提取失敗: {e}")
                
                # 正規化書名（去掉標點符號和空格）
                processed_title = re.sub(r'[^\w]', '', title) if title else ''
                
                # 書籍鏈結
                book_link = None
                try:
                    if title_element:
                        book_link = title_element.get_attribute('href')
                        # 金石堂的鏈結可能是相對路徑，需要補充完整 URL
                        if book_link and not book_link.startswith('http'):
                            book_link = f"https://www.kingstone.com.tw{book_link}"
                except Exception as e:
                    print(f"[除錯] 書籍 {index} 連結提取失敗: {e}")
                
                # 平台產品編號（從URL提取）
                production_id = ''
                if book_link:
                    # 金石堂的 URL 格式: /basic/2019461755150/?lid=...
                    match = re.search(r'/basic/(\d+)/', book_link)
                    if match:
                        production_id = match.group(1)
                
                # 作者
                author = ''
                try:
                    author_element = book.find_element(By.CSS_SELECTOR, '.modProAuthor a')
                    author = author_element.text.strip() if author_element else ''
                except:
                    pass
                
                # 除錯資訊
                if not (rank and title and book_link):
                    debug_msg = f"[除錯] 書籍 {index} 缺少資料: rank={rank}, title={title[:20] if title else None}, link={book_link[:50] if book_link else None}"
                    print(debug_msg)
                    log_entries.append(debug_msg)
                
                # 過濾條件：確保所有必要字段有效
                if rank and title and book_link:
                    # 按照博客來格式存入資料：
                    # ISBN, production_id, title, processed_title, Publisher, author, translator, 
                    # original_title, publish_date, fixed_price, category, url, Day_counts, 今天日期
                    all_books_data.append([
                        '',  # ISBN (空)
                        production_id,  # production_id
                        title,  # title
                        processed_title,  # processed_title (去掉標點符號和空格)
                        '',  # Publisher (空)
                        author,  # author
                        '',  # translator (空)
                        '',  # original_title (空)
                        '',  # publish_date (空)
                        '',  # fixed_price (空)
                        '',  # category (空）
                        book_link,  # url
                        '',  # Day_counts (空)
                        rank  # 今天的排名
                    ])
                    total_successful_books += 1
                    
                    # 日誌記錄成功條目
                    success_message = f"[成功] 排名: {rank}, 產品編號: {production_id}, 書名: {title}, 作者: {author}, 鏈結: {book_link}"
                    print(success_message)
                    log_entries.append(success_message)
                else:
                    total_failed_books += 1
                    error_message = f"[跳過] 書籍 {index}: 資料不完整，已略過。"
                    print(error_message)
                    log_entries.append(error_message)
                    
            except Exception as e:
                total_failed_books += 1
                error_message = f"[錯誤] 書籍 {index}: 解析失敗，錯誤訊息: {e}"
                print(error_message)
                log_entries.append(error_message)
        
        # 計算成功率
        success_rate = (total_successful_books / total_books * 100) if total_books > 0 else 0
        log_entries.append(f"\n統計結果:")
        log_entries.append(f"總抓取筆數: {total_books}")
        log_entries.append(f"成功筆數: {total_successful_books}")
        log_entries.append(f"失敗筆數: {total_failed_books}")
        log_entries.append(f"成功率: {success_rate:.2f}%\n")
        
        print("\n統計結果:")
        print(f"總抓取筆數: {total_books}")
        print(f"成功筆數: {total_successful_books}")
        print(f"失敗筆數: {total_failed_books}")
        print(f"成功率: {success_rate:.2f}%\n")
        
    except Exception as e:
        error_message = f"[嚴重錯誤] 頁面加載或解析失敗: {e}"
        print(error_message)
        log_entries.append(error_message)
    
    finally:
        # 關閉瀏覽器
        try:
            driver.quit()
            print("瀏覽器已關閉")
        except:
            pass
    
    # 將日誌記錄寫入日誌文件
    log_file.write("\n".join(log_entries))

# 將所有數據保存為一個 CSV 檔案
with open(csv_file_path, 'w', newline='', encoding='utf-8-sig') as file:
    writer = csv.writer(file)
    # 寫入標題行，最後一欄是今天的日期（與博客來格式一致）
    header = ['ISBN', 'production_id', 'title', 'processed_title', 'Publisher', 'author', 
              'translator', 'original_title', 'publish_date', 'fixed_price', 
              'category', 'url', 'Day_counts', today_column]
    writer.writerow(header)
    writer.writerows(all_books_data)

# 結束提示
print(f"金石堂總榜月排行數據已保存到 {csv_file_path}")
print(f"日誌已保存到 {log_file_path}")



開始抓取金石堂總榜月排行書籍數據...
正在載入頁面...
正在滾動頁面以加載所有書籍...
開始解析金石堂總榜月排行，共找到 96 本書
[成功] 排名: 1, 產品編號: 2019461755150, 書名: 40歲以前想達成的10件事 (首刷限定版) II, 作者: マミタ, 鏈結: https://www.kingstone.com.tw/basic/2019461755150/?lid=book_ranking_sell_ALL
[成功] 排名: 2, 產品編號: 2019461761779, 書名: 嫁入狼族～異種婚姻譚Ⅲ(特裝版)(完), 作者: 犬居葉菜, 鏈結: https://www.kingstone.com.tw/basic/2019461761779/?lid=book_ranking_sell_ALL
[成功] 排名: 3, 產品編號: 2019461755020, 書名: 40歲以前想達成的10件事 II, 作者: マミタ, 鏈結: https://www.kingstone.com.tw/basic/2019461755020/?lid=book_ranking_sell_ALL
[成功] 排名: 4, 產品編號: 2019461785171, 書名: 請不要放開這雙手(03)END 特典版, 作者: 咲本﨑, 鏈結: https://www.kingstone.com.tw/basic/2019461785171/?lid=book_ranking_sell_ALL
[成功] 排名: 5, 產品編號: 2019461780701, 書名: 難以承受的悸動是戀愛嗎？（１）, 作者: 空華みあ, 鏈結: https://www.kingstone.com.tw/basic/2019461780701/?lid=book_ranking_sell_ALL
[成功] 排名: 6, 產品編號: 2019461749821, 書名: 探索愛情的王冠男孩 (特裝版) II, 作者: まさき茉生, 鏈結: https://www.kingstone.com.tw/basic/2019461749821/?lid=book_ranking_sell_ALL
[成功] 排名: 7, 產品編號: 2018612499004, 書名: 世界上最透明的故事【

In [1]:
#eslite誠品實體書30天月暢銷榜單(每日更新)
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
import csv, os, re, time
from datetime import datetime

# 主網頁 URL
base_url = 'https://www.eslite.com/best-sellers/online/3?type=2'
today_date = datetime.now().strftime('%Y%m%d')
today_column = datetime.now().strftime('%-m/%-d') if os.name != 'nt' else datetime.now().strftime('%#m/%#d')

# 目錄
output_dir = r'C:/Users/HP/Desktop/project/book-ranking/ranking_result/eslite'
log_dir = r'C:/Users/HP/Desktop/project/book-ranking/實體書排行榜code/log'
os.makedirs(output_dir, exist_ok=True)
os.makedirs(log_dir, exist_ok=True)
csv_file_path = os.path.join(output_dir, f'eslite_all_categories_{today_date}.csv')
log_file_path = os.path.join(log_dir, f'eslite_plog_{today_date}.txt')

# 資料與統計
all_books_data = []
total_books = total_successful_books = total_failed_books = 0

# 啟動瀏覽器
options = Options()
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")
driver = webdriver.Chrome(options=options)

# 抓取
with open(log_file_path, 'w', encoding='utf-8') as log_file:
    log_entries = ["開始抓取誠品總榜書籍數據...\n"]
    print("開始抓取誠品總榜書籍數據...")
    
    try:
        # 先載入第一頁
        print(f"正在載入第一頁: {base_url}")
        driver.get(base_url)
        time.sleep(3)
        
        wait = WebDriverWait(driver, 15)
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "div.product-item")))
        time.sleep(2)
        
        for page_num in range(1, 6):  # 爬5頁
            # 改進的翻頁邏輯
            if page_num > 1:
                try:
                    print(f"正在點擊第 {page_num} 頁按鈕...")
                    
                    # 更精確的選擇器
                    page_button = wait.until(EC.element_to_be_clickable(
                        (By.XPATH, f"//div[@data-gid='pagination-page' and @data-position='{page_num}']//button")
                    ))
                    
                    # 記錄點擊前的第一本書標題
                    books_before = driver.find_elements(By.CSS_SELECTOR, "div.product-item")
                    first_book_before = books_before[0].find_element(By.CSS_SELECTOR, 'div.product-name span').text if books_before else ""
                    
                    # 點擊
                    driver.execute_script("arguments[0].click();", page_button)
                    time.sleep(5)  # 增加等待時間
                    
                    # 等待頁面更新
                    wait.until(EC.staleness_of(books_before[0]))
                    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "div.product-item")))
                    
                    # 驗證是否翻頁成功
                    books_after = driver.find_elements(By.CSS_SELECTOR, "div.product-item")
                    first_book_after = books_after[0].find_element(By.CSS_SELECTOR, 'div.product-name span').text if books_after else ""
                    
                    if first_book_before == first_book_after:
                        print(f"❌ 警告：翻頁可能失敗，第一本書相同: {first_book_before}")
                    else:
                        print(f"✅ 翻頁成功，第一本書從 '{first_book_before[:20]}...' 變為 '{first_book_after[:20]}...'")
                    
                except Exception as e:
                    print(f"❌ 點擊第 {page_num} 頁失敗: {e}")
                    continue
            
            books = driver.find_elements(By.CSS_SELECTOR, "div.product-item")
            print(f"第 {page_num} 頁找到 {len(books)} 本書")
            log_entries.append(f"第 {page_num} 頁找到 {len(books)} 本書")
            
            for index, book in enumerate(books, start=1):
                total_books += 1
                try:
                    # 修正排名計算：每頁20本書
                    rank = (page_num - 1) * 20 + index
                    title = author = publisher = publish_date = book_link = production_id = ''
                    
                    # 提取書籍連結和書名
                    try:
                        link_elem = book.find_element(By.CSS_SELECTOR, 'a[data-gid="title-link"]')
                        book_link = link_elem.get_attribute('href')
                        if book_link and not book_link.startswith('http'):
                            book_link = f"https://www.eslite.com{book_link}"
                        
                        if book_link:
                            match = re.search(r'/product/(\d+)', book_link)
                            if match:
                                production_id = match.group(1)
                        
                        title_elem = link_elem.find_element(By.CSS_SELECTOR, 'div.product-name span')
                        title = title_elem.text.strip()
                    except Exception as e:
                        print(f"    書籍 {index} 提取連結/書名失敗: {e}")
                    
                    # 提取作者、出版社、出版日期 - 修正容器選擇器
                    try:
                        # 關鍵修正：直接找到product-publish容器，不需要再找子div
                        publish_container = book.find_element(By.CSS_SELECTOR, 'div.item-wording-wrap div.product-publish.flex.flex-col')
                        debug_msg = f"    書籍 {index} 找到publish_container"
                        print(debug_msg)
                        log_entries.append(debug_msg)
                        
                        # 找作者
                        try:
                            author_elem = publish_container.find_element(By.CSS_SELECTOR, 'div.product-author a span')
                            author = author_elem.text.strip()
                            debug_msg = f"    書籍 {index} 找到作者: {author}"
                            print(debug_msg)
                            log_entries.append(debug_msg)
                        except Exception as e:
                            debug_msg = f"    書籍 {index} 作者失敗: {e}"
                            print(debug_msg)
                            log_entries.append(debug_msg)
                            pass
                        
                        # 找出版社 - 使用您提供的完整路徑
                        try:
                            publisher_elem = publish_container.find_element(By.CSS_SELECTOR, 'div.product-publisher.mr-1 a span')
                            publisher = publisher_elem.text.strip()
                            debug_msg = f"    書籍 {index} 找到出版社: {publisher}"
                            print(debug_msg)
                            log_entries.append(debug_msg)
                            # 如果出版社為空，嘗試其他選擇器
                            if not publisher:
                                try:
                                    # 嘗試沒有 .mr-1 的選擇器
                                    publisher_elem2 = publish_container.find_element(By.CSS_SELECTOR, 'div.product-publisher a span')
                                    publisher = publisher_elem2.text.strip()
                                    debug_msg = f"    書籍 {index} 備用出版社: {publisher}"
                                    print(debug_msg)
                                    log_entries.append(debug_msg)
                                except:
                                    debug_msg = f"    書籍 {index} 備用出版社也失敗"
                                    print(debug_msg)
                                    log_entries.append(debug_msg)
                        except Exception as e:
                            debug_msg = f"    書籍 {index} 出版社失敗: {e}"
                            print(debug_msg)
                            log_entries.append(debug_msg)
                            pass
                        
                        # 找出版日期 - 使用您提供的完整路徑
                        try:
                            date_span = publish_container.find_element(By.CSS_SELECTOR, 'div.product-date.mr-1 span:nth-child(2)')
                            text = date_span.text.strip()
                            debug_msg = f"    書籍 {index} 找到日期原文: {text}"
                            print(debug_msg)
                            log_entries.append(debug_msg)
                            
                            # 如果日期為空，嘗試其他選擇器
                            if not text or text == '|':
                                try:
                                    # 嘗試沒有 .mr-1 的選擇器
                                    date_span2 = publish_container.find_element(By.CSS_SELECTOR, 'div.product-date span:nth-child(2)')
                                    text = date_span2.text.strip()
                                    debug_msg = f"    書籍 {index} 備用日期原文: {text}"
                                    print(debug_msg)
                                    log_entries.append(debug_msg)
                                except:
                                    try:
                                        # 嘗試所有 span 元素，找包含"年"的
                                        date_container = publish_container.find_element(By.CSS_SELECTOR, 'div.product-date')
                                        all_spans = date_container.find_elements(By.TAG_NAME, 'span')
                                        for span in all_spans:
                                            span_text = span.text.strip()
                                            if span_text and span_text != '|' and '年' in span_text:
                                                text = span_text
                                                debug_msg = f"    書籍 {index} 找到年份日期: {text}"
                                                print(debug_msg)
                                                log_entries.append(debug_msg)
                                                break
                                    except:
                                        debug_msg = f"    書籍 {index} 所有日期方法都失敗"
                                        print(debug_msg)
                                        log_entries.append(debug_msg)
                            
                            if text and text != '|':
                                # 轉換日期格式: "2025年02月05日" -> "2025/2/5"
                                match = re.match(r'(\d{4})年(\d{1,2})月(\d{1,2})日', text)
                                if match:
                                    year, month, day = match.groups()
                                    publish_date = f"{year}/{int(month)}/{int(day)}"
                                else:
                                    publish_date = text
                                debug_msg = f"    書籍 {index} 轉換後日期: {publish_date}"
                                print(debug_msg)
                                log_entries.append(debug_msg)
                        except Exception as e:
                            debug_msg = f"    書籍 {index} 日期失敗: {e}"
                            print(debug_msg)
                            log_entries.append(debug_msg)
                            pass
                    except Exception as ex:
                        debug_msg = f"    書籍 {index} 整體失敗: {ex}"
                        print(debug_msg)
                        log_entries.append(debug_msg)
                        pass
                    
                    # 調試信息：顯示缺失的字段
                    missing = []
                    if not author: missing.append('作者')
                    if not publisher: missing.append('出版社')
                    if not publish_date: missing.append('出版日期')
                    if missing:
                        print(f"    ⚠️ 書籍 {index} ({title[:30]}...) 缺失: {', '.join(missing)}")
                    
                    processed_title = re.sub(r'[^\w]', '', title) if title else ''
                    
                    if title and book_link:
                        all_books_data.append([
                            '',  # ISBN
                            production_id,  # production_id
                            title,  # title
                            processed_title,  # processed_title
                            publisher,  # Publisher
                            author,  # author
                            '',  # translator
                            '',  # original_title
                            publish_date,  # publish_date
                            '',  # fixed_price
                            '',  # category (留空)
                            book_link,  # url
                            '',  # Day_counts
                            rank  # 排名
                        ])
                        total_successful_books += 1
                        msg = f"[成功] 排名: {rank}, 書名: {title}, 作者: {author}, 出版社: {publisher}, 出版日期: {publish_date}"
                        print(msg)
                        log_entries.append(msg)
                    else:
                        total_failed_books += 1
                        error_msg = f"[跳過] 第 {page_num} 頁書籍 {index}: 資料不完整"
                        print(error_msg)
                        log_entries.append(error_msg)
                        
                except Exception as e:
                    total_failed_books += 1
                    error_msg = f"[錯誤] 第 {page_num} 頁書籍 {index}: {e}"
                    print(error_msg)
                    log_entries.append(error_msg)
        
        success_rate = (total_successful_books / total_books * 100) if total_books > 0 else 0
        stats = f"\n統計結果:\n總筆數: {total_books}\n成功: {total_successful_books}\n失敗: {total_failed_books}\n成功率: {success_rate:.2f}%"
        print(stats)
        log_entries.append(stats)
        
    except Exception as e:
        error_msg = f"[嚴重錯誤] {e}"
        print(error_msg)
        log_entries.append(error_msg)
    finally:
        try:
            driver.quit()
            print("瀏覽器已關閉")
        except:
            pass
    
    log_file.write("\n".join(log_entries))

# 保存CSV
with open(csv_file_path, 'w', newline='', encoding='utf-8-sig') as file:
    writer = csv.writer(file)
    writer.writerow([
        'ISBN', 'production_id', 'title', 'processed_title', 'Publisher', 'author', 
        'translator', 'original_title', 'publish_date', 'fixed_price', 
        'category', 'url', 'Day_counts', today_column
    ])
    writer.writerows(all_books_data)

print(f"\n誠品總榜數據已保存到 {csv_file_path}")
print(f"日誌已保存到 {log_file_path}")

開始抓取誠品總榜書籍數據...
正在載入第一頁: https://www.eslite.com/best-sellers/online/3?type=2
第 1 頁找到 20 本書
    書籍 1 找到publish_container
    書籍 1 找到作者: 劉姿君,吉田修一
    書籍 1 找到出版社: 新經典圖文傳播有限公司
    書籍 1 找到日期原文: 2020年09月02日
    書籍 1 轉換後日期: 2020/9/2
[成功] 排名: 1, 書名: 國寶 上下 (2冊合售), 作者: 劉姿君,吉田修一, 出版社: 新經典圖文傳播有限公司, 出版日期: 2020/9/2
    書籍 2 找到publish_container
    書籍 2 找到作者: Peter Su
    書籍 2 找到出版社: 是日創意文化有限公司
    書籍 2 找到日期原文: 2025年11月30日
    書籍 2 轉換後日期: 2025/11/30
[成功] 排名: 2, 書名: 勇敢告別的人, 生活會獎勵一場新的開始 (限量親簽版/附碎光集Zine), 作者: Peter Su, 出版社: 是日創意文化有限公司, 出版日期: 2025/11/30
    書籍 3 找到publish_container
    書籍 3 找到作者: 黃山料
    書籍 3 找到出版社: 三采文化股份有限公司
    書籍 3 找到日期原文: 2025年12月01日
    書籍 3 轉換後日期: 2025/12/1
[成功] 排名: 3, 書名: 擁有過 就好: 告別青春的最後一封情書 (限量親簽版), 作者: 黃山料, 出版社: 三采文化股份有限公司, 出版日期: 2025/12/1
    書籍 4 找到publish_container
    書籍 4 找到作者: 查理．麥克斯,呂玉嬋,Charlie Mackesy
    書籍 4 找到出版社: 皇冠文化出版有限公司
    書籍 4 找到日期原文: 
    書籍 4 備用日期原文: 
    ⚠️ 書籍 4 (永遠記得: 男孩、鼴鼠、狐狸、馬與風暴 (限量燙金書籤版)...) 缺失: 出版日期
[成功] 排名: 4, 書名: 永遠記得: 男孩、鼴鼠、狐狸、馬與風暴 (限量燙金書籤版), 作者: 查